In [1]:
import pandas as pd
import numpy as np
import folium
from geopy.geocoders import Nominatim

In [2]:
county = pd.read_csv('https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/latimes-county-totals.csv')

county.head()

,date,county,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths
0,2020-01-26,Alameda,1,0,0,NaN,NaN
1,2020-01-26,Calaveras,9,0,0,NaN,NaN
2,2020-01-26,Contra Costa,13,0,0,NaN,NaN
3,2020-01-26,Humboldt,23,0,0,NaN,NaN
4,2020-01-26,Los Angeles,37,1,0,NaN,NaN


In [3]:
county['date'] = pd.to_datetime(county['date'])

grouped_by_county = county.iloc[county.groupby('county')['date'].agg(pd.Series.idxmax)].copy()

In [14]:
# Color code counties by confirmed case count. Red for counties with more than 300 confirmed cases and Blue otherwise.
grouped_by_county['color'] = grouped_by_county['confirmed_cases'].map(lambda x: 'red' if x > 300 else 'blue')

In [27]:
grouped_by_county.head()

,date,county,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths,latitude,longitude,color
3500,2020-05-11,Alameda,1,2101,71,37.0,0.0,37.609029,-121.899142,red
3501,2020-05-11,Alpine,3,1,0,0.0,0.0,38.589393,-119.834501,blue
3502,2020-05-11,Amador,5,8,0,0.0,0.0,38.449089,-120.591102,blue
3503,2020-05-11,Butte,7,20,0,2.0,0.0,39.651927,-121.585844,blue
3504,2020-05-11,Calaveras,9,13,0,0.0,0.0,38.255818,-120.498149,blue


In [5]:
geolocater = Nominatim(user_agent='jonathancheung12@yahoo.com')

latitudes = []
longitudes = []
for i in grouped_by_county['county']:
    location = geolocater.geocode(i+', California')
    latitudes.append(location.latitude)
    longitudes.append(location.longitude) 

In [6]:
grouped_by_county['latitude'] = np.array(latitudes)
grouped_by_county['longitude'] = np.array(longitudes)

In [31]:
# California's coordinates: 36.7783 lat 119.4179 long
cali_map = folium.Map(location = [36.7783, -119.4179], zoom_start = 6)

In [32]:
for county, color, lat, lon, death, cases in zip(grouped_by_county['county'], 
                                                 grouped_by_county['color'], 
                                                 grouped_by_county['latitude'], 
                                                 grouped_by_county['longitude'], 
                                                 grouped_by_county['deaths'], 
                                                 grouped_by_county['confirmed_cases']):
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        color = color,
        popup = f'{county} Deaths:{death} Cases:{cases}').add_to(cali_map)

In [33]:
cali_map